In [1]:
!pip install --upgrade pip

     |████████████████████████████████| 2.0 MB 6.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.2
    Uninstalling pip-21.2.2:
      Successfully uninstalled pip-21.2.2


In [2]:
!pip install aitextgen
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.3/572.3 kB 14.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 93.3 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 kB 115.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.0 MB/s eta 0:00:00 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 96.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.0/752.0 kB 121.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 87.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install ipywidgets

In [4]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Brownie Cookies

⅔ cup brown sugar
2 cup eggs
½ cup butter
⅔ cup chocolate
1 teaspoon vanilla extract
1 cup plain flour
1 teaspoon baking powder
½ teaspoon salt
⅔ cup dark chocolate chip
⅔ cup white chocolate chip


1. In a large bowl, whisk together the brown sugar and eggs.
2. Next, whisk in the butter, chocolate, and vanilla extract.
3. Fold in the flour, baking powder, and salt.
4. Finally mix in the chocolate chips.
5. Chill in the fridge for 30 minutes, until slightly set.
6. Preheat the oven to 350˚F (180°C) and line a baking tray with greaseproof paper.
7. Scoop golf ball-size dollops of cookie dough onto the baking tray, making sure you leave enough room for them to spread out.
8. Bake for 12 minutes, until crispy on top and slightly gooey in the middle.
9. Enjoy!



In [5]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe_fixed.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [6]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen
import numpy

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, tf_gpt2="124M", to_gpu=True)

In [7]:
# #Ai training disabled to prevent accidental overrides.

# file_name = "recipe_fixed.csv"
# train_tokenizer(file_name)
# data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
# ai.train(data, batch_size=50, num_steps=64000, generate_every=500, save_every=2000)
# ai.generate(prompt="cherry pie")

In [22]:
# Load the trained model
ai = aitextgen(model_folder="trained_model", tokenizer_file="aitextgen.tokenizer.json", to_gpu=True, tf_gpu="124M" block_size="64")
ai.generate(temperature=.4 , prompt="chicken fingers")

chicken fingers with the remaining herbs. Serve immediately.
10. Enjoy!
Build Your Own Pizza

1 large tortilla
3 tablespoons shredded mozzarella cheese
3 tablespoons ham
3 tablespoons marinara sauce




1. Cut each dough into tortillawis
1 cup recheese, and brout ham







1. Rolive avocketaisovegetable pitalsery


1. Papres maco
1. Frush avockimalsse together bungre bungre bungre bungu
1 egg wondillibanana

1 egg
1. Frushibanana
1. Dillibanana
2. Dibanana
2. Dibanana
2. Dibanana
2. Dibanana
2. Dibanana
2. Divanilla°Foribanan
